In [1]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
import tensorflow as tf
print(tf.__version__)


2024-12-14 09:56:14.395859: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-14 09:56:14.405626: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734166574.416687   10934 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734166574.419908   10934 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-14 09:56:14.431563: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.18.0


In [2]:
# Create a dataset  of the format
{
  "current_article_text": "Full text of Article A...",
  "possible_next_articles": ["Article B", "Article C", "Article D"],
  "target_article": "Article D",
  "final_target_article": "Article Z"
}


{'current_article_text': 'Full text of Article A...',
 'possible_next_articles': ['Article B', 'Article C', 'Article D'],
 'target_article': 'Article D',
 'final_target_article': 'Article Z'}

In [3]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"


In [4]:
import pandas as pd

In [5]:
article_data = pd.read_feather('Data/dataframes/article_dataframe.feather')

In [6]:
article_data['linkTarget']

# Ge max number of links
med_links = article_data['linkTarget'].apply(len).describe()

In [7]:
med_links

count    4604.000000
mean       26.038662
std        24.201491
min         0.000000
25%        11.000000
50%        19.000000
75%        33.000000
max       294.000000
Name: linkTarget, dtype: float64

In [8]:
# Load the paths
paths_df = pd.read_feather('Data/dataframes/paths.feather')
# only include successful paths
paths_df = paths_df[paths_df['finished']]

In [9]:
paths_df.columns

Index(['hashedIpAddress', 'timestamp', 'durationInSec', 'path', 'rating',
       'finished', 'failure_reason', 'start_article', 'target_article'],
      dtype='object')

In [10]:
import pandas as pd
from multiprocessing import Pool, cpu_count

# Define the function again if necessary
def replace_back_steps(path_str):
    articles = path_str.split(';')
    stack = []
    processed = []
    
    for article in articles:
        if article == '<':
            if stack:
                stack.pop()
                if stack:
                    last_article = stack[-1]
                    processed.append(last_article)
            # Else, skip appending anything
        else:
            stack.append(article)
            processed.append(article)
    
    return ';'.join(processed)

# Function to apply in parallel
def parallel_process(paths):
    with Pool(cpu_count()) as pool:
        processed_paths = pool.map(replace_back_steps, paths)
    return processed_paths

# Apply parallel processing
paths_df['processed_path'] = parallel_process(paths_df['path'].tolist())

# Verify the result
print(paths_df[['path', 'processed_path']])


                                                    path  \
0      14th_century;15th_century;16th_century;Pacific...   
1      14th_century;Europe;Africa;Atlantic_slave_trad...   
2      14th_century;Niger;Nigeria;British_Empire;Slav...   
3         14th_century;Renaissance;Ancient_Greece;Greece   
4      14th_century;Italy;Roman_Catholic_Church;HIV;R...   
...                                                  ...   
51313                   Yagan;Ancient_Egypt;Civilization   
51314  Yagan;Folklore;Brothers_Grimm;<;19th_century;C...   
51315  Yagan;Australia;England;France;United_States;T...   
51316  Yarralumla,_Australian_Capital_Territory;Austr...   
51317                            Ziad_Jarrah;Germany;Jew   

                                          processed_path  
0      14th_century;15th_century;16th_century;Pacific...  
1      14th_century;Europe;Africa;Atlantic_slave_trad...  
2      14th_century;Niger;Nigeria;British_Empire;Slav...  
3         14th_century;Renaissance;Ancient_

In [11]:

# Assuming `paths_df` and `article_data` are your existing DataFrames

# Step 1: Precompute Lookup Dictionaries
# Ensure 'linkTarget' is a list. If it's a string separated by a delimiter (e.g., ';'), split it accordingly.
# article_data['linkTarget'] = article_data['linkTarget'].apply(lambda x: x.split(';') if isinstance(x, str) else [])
article_text_dict = article_data.set_index('article')['plain_text'].to_dict()
article_links_dict = article_data.set_index('article')['linkTarget'].to_dict()

# Step 2: Collect Data in Lists
dataset_list = []

for idx, row in paths_df.iterrows():
    path_str = row['processed_path']
    final_target_article = row['target_article']
    
    # Split the path into individual articles
    current_path = path_str.split(';')
    path_length = len(current_path)
    
    # Iterate over each article in the current path
    for i, article_name in enumerate(current_path):
        # Retrieve the current article's text
        current_article_text = article_text_dict.get(article_name, "")
        
        # Retrieve the list of possible next articles
        possible_next_articles = article_links_dict.get(article_name, [])
        
        # Determine the target article
        if i + 1 < path_length:
            target_article = current_path[i + 1]
        else:
            target_article = final_target_article
        

        # Append the data point to the list
        dataset_list.append({
            'current_article_text': current_article_text,
            'possible_next_articles': possible_next_articles,
            'target_article': target_article,
            'final_target_article': final_target_article
        })

# Step 3: Bulk DataFrame Creation
dataset = pd.DataFrame(dataset_list)

# Optional: Free up memory by deleting the list
del dataset_list

# Optional: Reset index if needed
dataset.reset_index(drop=True, inplace=True)


In [12]:
dataset.columns

Index(['current_article_text', 'possible_next_articles', 'target_article',
       'final_target_article'],
      dtype='object')

In [13]:
# Assign unique IDs to articles

all_articles = article_data['article'].tolist()

article_to_id = {article: idx for idx, article in enumerate(sorted(all_articles))}
id_to_article = {idx: article for article, idx in article_to_id.items()}
id_dataset = pd.DataFrame()
id_dataset["possible_next_ids"] = dataset["possible_next_articles"].apply(lambda x: [article_to_id[article] for article in x])
id_dataset["target_id"] = dataset["target_article"].map(article_to_id)
id_dataset["final_target_id"] = dataset["final_target_article"].map(article_to_id)
id_dataset["current_article_text"] = dataset["current_article_text"]

In [14]:
id_dataset.head()

,possible_next_ids,target_id,final_target_id,current_article_text
0,"[3, 6, 73, 417, 597, 714, 886, 899, 1128, 1131...",6,137,#copyright\n\n14th century\n\n2007 Schools ...
1,"[0, 1, 2, 3, 4, 9, 11, 13, 18, 20, 21, 32, 33,...",9,137,#copyright\n\n15th century\n\n2007 Schools ...
2,"[0, 1, 2, 3, 4, 6, 11, 13, 18, 20, 21, 32, 33,...",3127,137,#copyright\n\n16th century\n\n2007 Schools ...
3,"[9, 11, 13, 18, 82, 122, 277, 318, 321, 351, 3...",371,137,#copyright\n\nPacific Ocean\n\n2007 Schools...
4,"[83, 87, 99, 122, 148, 160, 210, 226, 259, 277...",99,137,#copyright\n\nAtlantic Ocean\n\n2007 School...


In [15]:
dataset = id_dataset

In [16]:
import numpy as np

# Function to find the index of target_article in possible_next_ids
def find_target_index(possible_next_ids, target_id):
    try:
        return possible_next_ids.index(target_id)
    except ValueError:
        # If target_id not in possible_next_ids, return a default value, e.g., 0
        return 0

# Apply the function to create a 'label' column
dataset['label'] = dataset.apply(
    lambda row: find_target_index(row['possible_next_ids'], row['target_id']), axis=1
)

# Verify the labels
print(dataset[['possible_next_ids', 'target_id', 'label']].head())


                                   possible_next_ids  target_id  label
0  [3, 6, 73, 417, 597, 714, 886, 899, 1128, 1131...          6      1
1  [0, 1, 2, 3, 4, 9, 11, 13, 18, 20, 21, 32, 33,...          9      5
2  [0, 1, 2, 3, 4, 6, 11, 13, 18, 20, 21, 32, 33,...       3127     70
3  [9, 11, 13, 18, 82, 122, 277, 318, 321, 351, 3...        371     10
4  [83, 87, 99, 122, 148, 160, 210, 226, 259, 277...         99      2


In [17]:
for index, row in dataset.iterrows():
    # See if possible-next-ids contain target-id
    # if row['target_id'] in row['possible_next_ids']:
        # If yes, find the index and set it as the label
        # label = row['possible_next_ids'].index(row['target_id'])
        # print("exists")
    # Shuffling the possible_next_ids
    np.random.shuffle(row['possible_next_ids'])
    if not (row['target_id'] in row['possible_next_ids']):
        print("doesn't exist")
        # If not, add the target-id to the possible-next-ids in the correct spot and make sure the id is in the right spot in the sorted list
        row['possible_next_ids'].append(row['target_id'])
        # sort the list
        np.random.shuffle(row['possible_next_ids'])
        row['label'] = row['possible_next_ids'].index(row['target_id'])
    else:
        row['label'] = row['possible_next_ids'].index(row['target_id'])

doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn't exist
doesn'

In [18]:
# find the index of the target-id
for index, row in dataset.iterrows():
    label = row['possible_next_ids'].index(row['target_id'])
    if label == 0:
        if row['target_id'] != row['possible_next_ids'][0]:
            print("Error")
    if row['target_id'] != row['possible_next_ids'][label]:
        print("Error")
        


In [19]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_POSSIBLE_NEXT = 312  # Adjust based on your dataset

# Pad possible_next_ids
dataset['possible_next_ids_padded'] = pad_sequences(
    dataset['possible_next_ids'],
    maxlen=MAX_POSSIBLE_NEXT,
    padding='post',
    truncating='post',
    value=0  # Assuming 0 is the padding ID
).tolist()

# If possible_next_ids are already padded during preprocessing, skip this step


In [20]:
dataset.head()

,possible_next_ids,target_id,final_target_id,current_article_text,label,possible_next_ids_padded
0,"[886, 1385, 3, 2071, 3456, 1379, 2164, 2029, 6...",6,137,#copyright\n\n14th century\n\n2007 Schools ...,1,"[886, 1385, 3, 2071, 3456, 1379, 2164, 2029, 6..."
1,"[9, 32, 13, 3, 902, 3821, 4141, 2236, 1906, 33...",9,137,#copyright\n\n15th century\n\n2007 Schools ...,5,"[9, 32, 13, 3, 902, 3821, 4141, 2236, 1906, 33..."
2,"[3069, 3315, 2440, 56, 1318, 54, 687, 4498, 14...",3127,137,#copyright\n\n16th century\n\n2007 Schools ...,70,"[3069, 3315, 2440, 56, 1318, 54, 687, 4498, 14..."
3,"[2662, 3665, 2411, 3143, 4287, 3898, 2218, 303...",371,137,#copyright\n\nPacific Ocean\n\n2007 Schools...,10,"[2662, 3665, 2411, 3143, 4287, 3898, 2218, 303..."
4,"[654, 883, 1585, 4234, 2714, 3801, 4258, 797, ...",99,137,#copyright\n\nAtlantic Ocean\n\n2007 School...,2,"[654, 883, 1585, 4234, 2714, 3801, 4258, 797, ..."


In [21]:
dataset.label.value_counts()

label
0      69513
4       9854
2       9648
3       9528
1       9486
       ...  
286        1
255        1
236        1
237        1
254        1
Name: count, Length: 286, dtype: int64

In [29]:
for index, row in dataset.iterrows():
    row['label'] = row['possible_next_ids_padded'].index(row['target_id'])
    # print(row['label'])
    # Assign to dataset
    dataset.at[index, 'label'] = row['label']

In [30]:
dataset.head()

,possible_next_ids,target_id,final_target_id,current_article_text,label,possible_next_ids_padded
0,"[886, 1385, 3, 2071, 3456, 1379, 2164, 2029, 6...",6,137,#copyright\n\n14th century\n\n2007 Schools ...,8,"[886, 1385, 3, 2071, 3456, 1379, 2164, 2029, 6..."
1,"[9, 32, 13, 3, 902, 3821, 4141, 2236, 1906, 33...",9,137,#copyright\n\n15th century\n\n2007 Schools ...,0,"[9, 32, 13, 3, 902, 3821, 4141, 2236, 1906, 33..."
2,"[3069, 3315, 2440, 56, 1318, 54, 687, 4498, 14...",3127,137,#copyright\n\n16th century\n\n2007 Schools ...,27,"[3069, 3315, 2440, 56, 1318, 54, 687, 4498, 14..."
3,"[2662, 3665, 2411, 3143, 4287, 3898, 2218, 303...",371,137,#copyright\n\nPacific Ocean\n\n2007 Schools...,69,"[2662, 3665, 2411, 3143, 4287, 3898, 2218, 303..."
4,"[654, 883, 1585, 4234, 2714, 3801, 4258, 797, ...",99,137,#copyright\n\nAtlantic Ocean\n\n2007 School...,39,"[654, 883, 1585, 4234, 2714, 3801, 4258, 797, ..."


In [31]:
dataset.label.value_counts()

label
0      12746
2      12569
1      12527
3      11993
4      11838
       ...  
289       23
227       23
276       20
278       19
294        2
Name: count, Length: 295, dtype: int64

In [32]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
from tqdm import tqdm

def batch_preprocess_and_save_dataset(
    dataset, 
    article_to_id, 
    tokenizer, 
    output_dir, 
    max_length=512, 
    max_possible_next=10, 
    batch_size=1000
):
    """
    Preprocess the dataset in batches and save to disk incrementally.

    Parameters:
    - dataset (pd.DataFrame): DataFrame containing the data points.
    - article_to_id (dict): Dictionary mapping article names to IDs.
    - tokenizer (transformers.PreTrainedTokenizerFast): Fast tokenizer instance.
    - output_dir (str): Directory to save processed batches.
    - max_length (int): Maximum length for tokenization.
    - max_possible_next (int): Maximum number of possible next articles.
    - batch_size (int): Number of samples to process in each batch.
    """
    num_samples = len(dataset)
    batch_num = 0

    # Iterate over the dataset in batches with a progress bar
    for start in tqdm(range(0, num_samples, batch_size), desc='Processing Batches'):
        end = min(start + batch_size, num_samples)
        batch = dataset.iloc[start:end]

        # 1. Tokenize 'current_article_text'
        texts = batch['current_article_text'].tolist()
        encoded = tokenizer(
            texts,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='np'  # Return as NumPy arrays
        )
        input_ids_batch = encoded['input_ids']
        attention_mask_batch = encoded['attention_mask']

        # 2. Map 'possible_next_articles' to IDs with padding/truncation
        # possible_next_articles = batch['possible_next_article'].tolist()
        # possible_next_ids_batch = np.array([
        #     [article_to_id.get(article, 0) for article in articles[:max_possible_next]] + [0]*(max_possible_next - len(articles)) 
        #     if len(articles) < max_possible_next else [article_to_id.get(article, 0) for article in articles[:max_possible_next]]
        #     for articles in possible_next_articles
        # ], dtype=np.int32)

        posisble_next_ids_batch = np.array(batch['possible_next_ids_padded'].tolist(), dtype=np.int32)

        # 3. Map 'target_article' and 'final_target_article' to IDs
        # target_ids_batch = batch['target_article'].map(lambda x: article_to_id.get(x, 0)).values
        # final_target_ids_batch = batch['final_target_article'].map(lambda x: article_to_id.get(x, 0)).values


        target_ids_batch = batch['target_id'].values
        final_target_ids_batch = batch['final_target_id'].values

        # Combine into a dictionary
        processed_batch = {
            'input_ids': input_ids_batch,
            'attention_mask': attention_mask_batch,
            'possible_next_ids': posisble_next_ids_batch,
            'target_id': np.array(target_ids_batch, dtype=np.int32),
            'final_target_id': np.array(final_target_ids_batch, dtype=np.int32),
        }

        # Save batch to disk using pickle
        batch_path = f"{output_dir}/batch_{batch_num}.pkl"
        with open(batch_path, 'wb') as f:
            pickle.dump(processed_batch, f)

        batch_num += 1

    print(f"All batches saved to {output_dir}")


In [33]:
from transformers import DistilBertTokenizerFast

# Use the fast tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')




batch_preprocess_and_save_dataset(
    dataset=dataset,
    article_to_id=article_to_id,
    tokenizer=tokenizer,
    output_dir='./processed_batches_ids_312_shuffled',
    max_length=512,
    max_possible_next=312,
    batch_size=1000
)


Processing Batches: 100%|██████████| 347/347 [16:14<00:00,  2.81s/it]


All batches saved to ./processed_batches_ids_312_shuffled


In [34]:
import tensorflow as tf
import pickle
import os
import random
import numpy as np

def load_sampled_batches_to_tf_dataset(batch_dir, sample_size=5):
    """
    Load a random sample of batches from disk and create a TensorFlow dataset.

    Parameters:
    - batch_dir (str): Directory containing the saved batches.
    - sample_size (int): Number of batch files to sample.

    Returns:
    - tf.data.Dataset: TensorFlow dataset containing sampled data.
    """
    # List all batch files in the directory
    all_files = [f for f in sorted(os.listdir(batch_dir)) if f.endswith('.pkl')]

    # Sample a subset of the files
    sampled_files = random.sample(all_files, min(sample_size, len(all_files)))

    # Placeholder lists for TensorFlow dataset creation
    input_ids_list = []
    attention_mask_list = []
    possible_next_ids_list = []
    final_target_ids_list = []
    target_ids_list = []

    # Load sampled batches
    for filename in sampled_files:
        with open(os.path.join(batch_dir, filename), 'rb') as f:
            batch = pickle.load(f)
            
            # Append to lists
            input_ids_list.append(batch['input_ids'])
            attention_mask_list.append(batch['attention_mask'])
            possible_next_ids_list.append(batch['possible_next_ids'])
            final_target_ids_list.append(batch['final_target_id'])
            target_ids_list.append(batch['target_id'])

    # Concatenate sampled batches into NumPy arrays
    input_ids = np.concatenate(input_ids_list, axis=0)
    attention_mask = np.concatenate(attention_mask_list, axis=0)
    possible_next_ids = np.concatenate(possible_next_ids_list, axis=0)
    final_target_ids = np.concatenate(final_target_ids_list, axis=0)
    target_ids = np.concatenate(target_ids_list, axis=0)

    # Reshape final_target_ids to (batch_size, 1)
    final_target_ids = final_target_ids.reshape(-1, 1)
    target_ids = target_ids.reshape(-1, 1)

    # Create a TensorFlow dataset
    tf_dataset = tf.data.Dataset.from_tensor_slices(({
        'input_ids': tf.constant(input_ids, dtype=tf.int32),
        'attention_mask': tf.constant(attention_mask, dtype=tf.int32),
        'possible_next_ids': tf.constant(possible_next_ids, dtype=tf.int32),
        'final_target_id': tf.constant(final_target_ids, dtype=tf.int32)
    }, tf.constant(target_ids, dtype=tf.int32)))

    return tf_dataset

# Load a sampled dataset and apply transformations
batch_dir = './processed_batches_ids_312'
sample_size = 100  # Adjust based on your memory capacity

tf_dataset = load_sampled_batches_to_tf_dataset(batch_dir, sample_size=sample_size)

# Shuffle, batch, and prefetch
BATCH_SIZE = 32
BUFFER_SIZE = 10000

tf_dataset = tf_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


I0000 00:00:1734167863.138680   10934 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5520 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [22]:
tf_dataset.element_spec

({'input_ids': TensorSpec(shape=(None, 512), dtype=tf.int32, name=None),
  'attention_mask': TensorSpec(shape=(None, 512), dtype=tf.int32, name=None),
  'possible_next_ids': TensorSpec(shape=(None, 312), dtype=tf.int32, name=None),
  'final_target_id': TensorSpec(shape=(None, 1), dtype=tf.int32, name=None)},
 TensorSpec(shape=(None, 1), dtype=tf.int32, name=None))

I want to teach a model how to play the wikipedia game, one article and "final target" article at a time. I have a batched dataset that looks like the following:
({'input_ids': TensorSpec(shape=(None, 512), dtype=tf.int32, name=None),
  'attention_mask': TensorSpec(shape=(None, 512), dtype=tf.int32, name=None),
  'possible_next_ids': TensorSpec(shape=(None, 32), dtype=tf.int32, name=None),
  'final_target_id': TensorSpec(shape=(None, 1), dtype=tf.int32, name=None)},
 TensorSpec(shape=(None, 1), dtype=tf.int32, name=None))

Where "input_ids" are the first 512 tokens from the article text by distil_bert_uncased, and attention_mask is the mask output from calling the encoder, like so

        # 1. Tokenize 'current_article_text'
        texts = batch['current_article_text'].tolist()
        encoded = tokenizer(
            texts,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='np'  # Return as NumPy arrays
        )
        input_ids_batch = encoded['input_ids']
        attention_mask_batch = encoded['attention_mask']

possible_next_id's are the possible output articles of the current article, in numeric id form. these are the articles the model can choose from when outputting a prediciton for the next article to go to.
final_target_id's are not the output of the model, but rather the final goal of the current round of playing the wikipeedia game. The model should be trying to get to this article as fast as possible.

Finaly, the second term is the "correct" choice, or rather what a human chose as the next article in a successful round of playing the game.

In [23]:
for batch in tf_dataset.take(1):
    inputs, targets = batch
    print({k: v.shape for k, v in inputs.items()})
    print(f"Targets shape: {targets.shape}")


{'input_ids': TensorShape([32, 512]), 'attention_mask': TensorShape([32, 512]), 'possible_next_ids': TensorShape([32, 312]), 'final_target_id': TensorShape([32, 1])}
Targets shape: (32, 1)


2024-12-13 11:07:25.265121: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [24]:
for batch in tf_dataset.take(1):

    print(batch[0])
    print(batch[1])    

{'input_ids': <tf.Tensor: shape=(32, 512), dtype=int32, numpy=
array([[  101,  1001,  9385, ...,  2048,  6405,   102],
       [  101,  1001,  9385, ..., 18657, 23517,   102],
       [  101,  1001,  9385, ...,  2198,  1039,   102],
       ...,
       [  101,  1001,  9385, ...,  1012,  2761,   102],
       [  101,  1001,  9385, ...,  2002,  2001,   102],
       [  101,  1001,  9385, ...,  1149,  2094,   102]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(32, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int32)>, 'possible_next_ids': <tf.Tensor: shape=(32, 312), dtype=int32, numpy=
array([[ 193,  261,  277, ...,    0,    0,    0],
       [  87,   96,   99, ...,    0,    0,    0],
       [  32,  227,  369, ...,    0,    0,    0],
       ...,
       [ 571, 1080, 1427, ...,    0,    0,    0],
    

2024-12-13 11:07:25.348689: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [25]:
# Describe the dataset
tf_dataset.element_spec

({'input_ids': TensorSpec(shape=(None, 512), dtype=tf.int32, name=None),
  'attention_mask': TensorSpec(shape=(None, 512), dtype=tf.int32, name=None),
  'possible_next_ids': TensorSpec(shape=(None, 312), dtype=tf.int32, name=None),
  'final_target_id': TensorSpec(shape=(None, 1), dtype=tf.int32, name=None)},
 TensorSpec(shape=(None, 1), dtype=tf.int32, name=None))

In [26]:
for batch in tf_dataset.take(1):
    inputs, targets = batch
    print(inputs)
    for key, value in inputs.items():
        print(f"{key}: {value.shape}")
    print(f"Targets: {targets.shape}")


{'input_ids': <tf.Tensor: shape=(32, 512), dtype=int32, numpy=
array([[  101,  1001,  9385, ...,  1999,  4662,   102],
       [  101,  1001,  9385, ..., 18447, 21945,   102],
       [  101,  1001,  9385, ...,  4048,  6199,   102],
       ...,
       [  101,  1001,  9385, ...,  3574,  1000,   102],
       [  101,  1001,  9385, ...,  1012,  2174,   102],
       [  101,  1001,  9385, ...,  2323,  2022,   102]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(32, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int32)>, 'possible_next_ids': <tf.Tensor: shape=(32, 312), dtype=int32, numpy=
array([[122, 351, 387, ...,   0,   0,   0],
       [315, 341, 580, ...,   0,   0,   0],
       [247, 360, 375, ...,   0,   0,   0],
       ...,
       [  1,   6,   9, ...,   0,   0,   0],
       [159, 628, 653, ..., 

In [35]:

def id_to_index(features, label_id):
    # label_id is of shape (batch_size, 1), we need to squeeze it
    label_id = tf.squeeze(label_id, axis=1)  # Now shape (batch_size,)

    possible_ids = features['possible_next_ids']  # shape: (batch_size, 32)

    # Create a boolean mask for each position: True where possible_ids == label_id
    matches = tf.equal(possible_ids, tf.expand_dims(label_id, axis=1))  # (batch_size, 32)

    # Convert boolean to int to use argmax
    matches_int = tf.cast(matches, tf.int32)

    # If the label_id is guaranteed to be in possible_next_ids, argmax will find the correct index.
    # argmax on a row of all zeros would yield 0, so ensure that there's always a match.
    index = tf.argmax(matches_int, axis=1)  # (batch_size,)

    # index is now the integer index of the label_id in possible_next_ids.
    return features, index


# Apply this transformation to your dataset:
tf_dataset_indexed = tf_dataset.map(id_to_index)

# Now tf_dataset_indexed should yield ((features), label_index) 
# where label_index is in [0, num_candidates-1].

In [28]:
for batch in tf_dataset_indexed.take(1):
    inputs, label_index = batch
    print(inputs)
    print(f"Label Index: {label_index}")

{'input_ids': <tf.Tensor: shape=(32, 512), dtype=int32, numpy=
array([[  101,  1001,  9385, ..., 20860,  1010,   102],
       [  101,  1001,  9385, ...,  3142,  2661,   102],
       [  101,  1001,  9385, ...,  1010, 12121,   102],
       ...,
       [  101,  1001,  9385, ...,  1996,  2088,   102],
       [  101,  1001,  9385, ..., 13031, 10888,   102],
       [  101,  1001,  9385, ...,  2642,  3088,   102]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(32, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int32)>, 'possible_next_ids': <tf.Tensor: shape=(32, 312), dtype=int32, numpy=
array([[ 18, 358, 367, ...,   0,   0,   0],
       [ 32,  36,  44, ...,   0,   0,   0],
       [122, 170, 223, ...,   0,   0,   0],
       ...,
       [122, 261, 308, ...,   0,   0,   0],
       [ 70, 116, 303, ..., 

2024-12-13 11:07:25.687643: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [36]:
import tensorflow as tf
from transformers import TFDistilBertModel

# Load DistilBERT
distilbert = TFDistilBertModel.from_pretrained('distilbert-base-uncased')




Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [37]:

# Example hyperparameters
embedding_dim = 64
max_length = 512
num_candidates = 312
num_article_ids = 500000  # Adjust based on your dataset

class WikiNextArticleModel(tf.keras.Model):
    def __init__(self, distilbert, embedding_dim, num_article_ids, num_candidates):
        super().__init__()
        self.distilbert = distilbert
        self.final_target_embedding = tf.keras.layers.Embedding(num_article_ids, embedding_dim)
        self.candidate_embedding = tf.keras.layers.Embedding(num_article_ids, embedding_dim)
        self.query_dense_1 = tf.keras.layers.Dense(128, activation='relu')
        self.query_dense_2 = tf.keras.layers.Dense(embedding_dim)
        self.softmax = tf.keras.layers.Softmax()
    
    def call(self, inputs, training=False):
        # inputs is a dict: {'input_ids':..., 'attention_mask':..., 'possible_next_ids':..., 'final_target_id':...}
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        possible_next_ids = inputs['possible_next_ids']
        final_target_id = inputs['final_target_id']
        
        outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask, training=training)
        sequence_output = outputs.last_hidden_state
        cls_rep = sequence_output[:, 0, :]

        final_target_emb = tf.squeeze(self.final_target_embedding(final_target_id), axis=1)
        candidates_emb = self.candidate_embedding(possible_next_ids)
        
        combined_rep = tf.concat([cls_rep, final_target_emb], axis=-1)
        query = self.query_dense_1(combined_rep)
        query = self.query_dense_2(query)

        query_expanded = tf.expand_dims(query, axis=1)
        logits = tf.reduce_sum(query_expanded * candidates_emb, axis=-1)
        return self.softmax(logits)


In [31]:
# # Example validation:
# # extract labels from a batch
# for batch in tf_dataset_indexed.take(1):
#     inputs, targets = batch
#     labels = targets
#     tf.debugging.assert_less(labels, num_candidates, message="Labels out of candidate range")
#     tf.debugging.assert_greater_equal(labels, 0, message="Labels contain negative indices")

In [32]:
for features, label in tf_dataset_indexed.take(1):
    print({k: (v.shape, v.dtype, tf.reduce_min(v), tf.reduce_max(v)) for k,v in features.items()})
    print("Label: ", label.shape, label.dtype, tf.reduce_min(label), tf.reduce_max(label))


{'input_ids': (TensorShape([32, 512]), tf.int32, <tf.Tensor: shape=(), dtype=int32, numpy=100>, <tf.Tensor: shape=(), dtype=int32, numpy=30265>), 'attention_mask': (TensorShape([32, 512]), tf.int32, <tf.Tensor: shape=(), dtype=int32, numpy=1>, <tf.Tensor: shape=(), dtype=int32, numpy=1>), 'possible_next_ids': (TensorShape([32, 312]), tf.int32, <tf.Tensor: shape=(), dtype=int32, numpy=0>, <tf.Tensor: shape=(), dtype=int32, numpy=4597>), 'final_target_id': (TensorShape([32, 1]), tf.int32, <tf.Tensor: shape=(), dtype=int32, numpy=66>, <tf.Tensor: shape=(), dtype=int32, numpy=4505>)}
Label:  (32,) <dtype: 'int64'> tf.Tensor(0, shape=(), dtype=int64) tf.Tensor(141, shape=(), dtype=int64)


In [39]:

# Instantiate the model
model = WikiNextArticleModel(
    distilbert=distilbert,
    embedding_dim=embedding_dim,
    num_article_ids=num_article_ids,
    num_candidates=num_candidates
)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

# tf_dataset should yield (({'input_ids': ..., 'attention_mask': ..., 'possible_next_ids': ..., 'final_target_id': ...}), label)
model.fit(tf_dataset_indexed, epochs=20)

Epoch 1/20


W0000 00:00:1734168212.609179   20205 assert_op.cc:38] Ignoring Assert operator wiki_next_article_model_1_1/tf_distil_bert_model/distilbert/embeddings/assert_less/Assert/Assert


3125/3125 ━━━━━━━━━━━━━━━━━━━━ 827s 262ms/step - accuracy: 0.0948 - loss: 3.7234
Epoch 2/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 822s 263ms/step - accuracy: 0.1431 - loss: 3.4042
Epoch 3/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 822s 263ms/step - accuracy: 0.1496 - loss: 3.3725
Epoch 4/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 855s 261ms/step - accuracy: 0.1542 - loss: 3.3423
Epoch 5/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 820s 263ms/step - accuracy: 0.1585 - loss: 3.3097
Epoch 6/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 822s 263ms/step - accuracy: 0.1680 - loss: 3.2772
Epoch 7/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 822s 263ms/step - accuracy: 0.1760 - loss: 3.2405
Epoch 8/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 826s 264ms/step - accuracy: 0.1822 - loss: 3.2089
Epoch 9/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 819s 262ms/step - accuracy: 0.1885 - loss: 3.1818
Epoch 10/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 818s 262ms/step - accuracy: 0.1962 - loss: 3.1505
Epoch 11/20
3125/3125 ━━━━━━━━━━━━━━━━━━━━ 838s 268ms/step - accuracy: 0.1999 - loss: 3.12

In [41]:
model.save('model_312_shuffled.keras')

: 

In [ ]:
# Display predictions of model on a datapoint.

for features, label in tf_dataset_indexed.take(1):
    predictions = model(features)
    print(f"Predictions shape: {predictions.shape}")
    print(f"Predictions: {predictions}")
    # Decode the predictions to get the predicted index
    predicted_index = tf.argmax(predictions, axis=1)
    print(f"Predicted Index: {predicted_index}")
    print(f"True Index: {label}")

    